In [1]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma


Note: you may need to restart the kernel to use updated packages.


In [9]:
from typing import Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document

class CustomDocumentLoader(BaseLoader):
    """An example document loader that reads a file line by line."""

    def __init__(self, file_path: str) -> None:
        """Initialize the loader with a file path.

        Args:
            file_path: The path to the file to load.
        """
        self.file_path = file_path

    def lazy_load(self) -> Iterator[Document]:  # <-- Does not take any arguments
        """A lazy loader that reads a file line by line.

        When you're implementing lazy load methods, you should use a generator
        to yield documents one by one.
        """
        with open(self.file_path, encoding="utf-8") as f:
            line_number = 0
            for line in f:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1


In [12]:
loader = CustomDocumentLoader('src/chunk/sentences_1726807436568.txt')
requirements = loader.load()

In [13]:
%pip install -qU langchain-huggingface

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

Note: you may need to restart the kernel to use updated packages.


In [14]:
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=requirements, embedding=embeddings_model)

/home/ethan/Documents/code/icl-rag/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/ethan/Documents/code/icl-rag/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


What's the vector store actually giving me for a Requirements request?

In [17]:
request = "The Gateway shall provide a minimum of 24kW for system use when SEP is active."
sim = vectorstore.similarity_search(request, k=)
print(sim)

[Document(metadata={'line_number': 525, 'source': 'src/chunk/sentences_1726807436568.txt'}, page_content='The Gateway shall provide a minimum of 24kW for system use when SEP is active.\n'), Document(metadata={'line_number': 494, 'source': 'src/chunk/sentences_1726807436568.txt'}, page_content='The Gateway shall provide a minimum of 32kW for Gateway use when SEP is inactive.\n'), Document(metadata={'line_number': 531, 'source': 'src/chunk/sentences_1726807436568.txt'}, page_content='The Gateway shall reserve a minimum of 4kW power for utilization use.\n'), Document(metadata={'line_number': 520, 'source': 'src/chunk/sentences_1726807436568.txt'}, page_content='The Gateway shall provide energy storage capacity capable of supplying a minimum of 32kW during non-insolation periods of at least 1.5 hrs.\n'), Document(metadata={'line_number': 527, 'source': 'src/chunk/sentences_1726807436568.txt'}, page_content='The 24kW electrical power value represents the minimum required continuous transfer